In [3]:
import pandas as pd
import pathlib
import duckdb

# --- Configuração ---
PASTA_STAGING = pathlib.Path("dados_catalogo") / "staging"
CAMINHO_PARQUETS = str(PASTA_STAGING / "*.parquet")
ARQUIVO_RELATORIO_PLATAFORMAS = 'relatorio_plataformas.csv'

print("Iniciando a análise de plataformas e períodos...")

# --- Lógica Principal com DuckDB ---
try:
    con = duckdb.connect(database=':memory:', read_only=False)
    
    query_plataformas = f"""
    SELECT
        Platform_Name,
        COUNT(DISTINCT data_ref) AS semanas_presente,
        MIN(data_ref) AS primeira_aparicao,
        MAX(data_ref) AS ultima_aparicao,
        SUM(1) AS total_registros
    FROM
        read_parquet('{CAMINHO_PARQUETS}', union_by_name=true)
    WHERE
        Platform_Name IS NOT NULL
    GROUP BY
        Platform_Name
    """
    
    df_plataformas = con.execute(query_plataformas).fetch_df()
    con.close()
    
    if df_plataformas.empty:
        print("Nenhum dado encontrado nos arquivos Parquet. Verifique a pasta /staging.")
    else:
        # --- MELHORIA APLICADA AQUI ---
        df_plataformas['media_registros_por_semana'] = (
            df_plataformas['total_registros'] / df_plataformas['semanas_presente']
        ).fillna(0).round(0).astype(int)

        df_plataformas = df_plataformas.sort_values(by='media_registros_por_semana', ascending=False)
        
        df_plataformas = df_plataformas[[
            'Platform_Name',
            'media_registros_por_semana',
            'total_registros',
            'semanas_presente',
            'primeira_aparicao',
            'ultima_aparicao'
        ]]
        
        # Salva o relatório em CSV no padrão Excel-BR (sep=';', dec=',')
        df_plataformas.to_csv(
            ARQUIVO_RELATORIO_PLATAFORMAS,
            index=False,
            sep=';',
            decimal=',',
            encoding='utf-8-sig'
        )
        print(f"\n✔ Relatório completo e ordenado salvo em: '{ARQUIVO_RELATORIO_PLATAFORMAS}'")

        print("\n--- Sumário de Plataformas Encontradas (ordenado por média de registros/semana) ---")
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_rows', 100)
        print(df_plataformas)
        pd.reset_option('display.max_rows')
        pd.reset_option('display.width')

except Exception as e:
    print(f"\nOcorreu um erro durante a análise: {e}")
    print("Verifique se há arquivos na pasta 'dados_catalogo/staging' e se os nomes de colunas ('Platform_Name', 'data_ref') estão corretos.")


Iniciando a análise de plataformas e períodos...

✔ Relatório completo e ordenado salvo em: 'relatorio_plataformas.csv'

--- Sumário de Plataformas Encontradas (ordenado por média de registros/semana) ---
          Platform_Name  media_registros_por_semana  total_registros  semanas_presente primeira_aparicao ultima_aparicao
107       Archivio Luce                       68816        3784895.0                55        2024-07-11      2025-08-28
37            Claro TV+                       42810        2354539.0                55        2024-07-11      2025-08-28
125     NBA League Pass                       29237        1637259.0                56        2024-07-11      2025-08-28
45   Amazon Prime Video                       27645        1520462.0                55        2024-07-11      2025-08-28
85            Vivo Play                       19630        1079671.0                55        2024-07-11      2025-08-28
..                  ...                         ...              ... 